# Part 2. Topic modelling

In [1]:
import bz2, json
import tqdm
import pandas as pd
import numpy as np

In [2]:
path = 'banki_responses.json.bz2'

with bz2.open(path, 'rt') as bzinput:
    lines = []
    for i, cont in tqdm.tqdm(enumerate(bzinput)):
        tweets = json.loads(cont)
        lines.append(tweets)

201030it [02:10, 1539.21it/s]


In [3]:
df = pd.DataFrame(lines)
df.head(5)

,city,rating_not_checked,title,num_comments,bank_license,author,bank_name,datetime,text,rating_grade
0,г. Воронеж,True,Ипотека на 5+,0,лицензия № 1623,e.novikova,ВТБ 24,2015-06-08 11:06:56,Здравствуйте! Хотелось бы выразить благодарно...,5.0
1,г. Казань,True,ЗВОНКИ СОСЕДЯМ,0,лицензия № 2289,KZN\Vorontsova_NA,Русский Стандарт,2015-06-06 18:17:52,Уважаемые представители департамента по работе...,2.0
2,г. Санкт-Петербург,True,Ложная информация!!!!,0,лицензия № 2307,Evgenia15,Союз,2015-06-07 19:08:33,Здравствуйте. Столкнулась с такой ситуацией. в...,2.0
3,г. Москва,True,Банкомат не зачислил деньги на счет,1,лицензия № 2557,dolgov.work,Ситибанк,2015-06-07 19:22:00,1 июня 2015 в Москве в отделении по адресу Смо...,1.0
4,г. Волгоград,True,Невозможно закрыть карту 2(два)!!!,0,лицензия № 3354,copLine,Ренессанс Кредит,2015-06-08 14:54:55,История очень долгая! Пришел я в банк (это раз...,1.0


## 1. Постройте несколько тематических моделей коллекции документов с разным числом тем. 

- Приведите примеры понятных (интерпретируемых) тем.

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import pymorphy3, re, nltk
from gensim.corpora import Dictionary
from gensim.models import TfidfModel

In [6]:
morph = pymorphy3.MorphAnalyzer()
ru_words = re.compile(r"[А-Яа-я]+")
stops = nltk.corpus.stopwords.words('russian')

def preprocess_text(text):
    
    def check_stop_words(text):
        return filter(lambda x: x not in stops, text)
    
    def lemmatize(text):
        return map(lambda x: morph.parse(x)[0].normal_form, text)
    
    def words_only(text):
        return ru_words.findall(text.lower())

    return ' '.join(lemmatize(check_stop_words(words_only(text))))

In [7]:
texts = df['text'][:5000].apply(preprocess_text)

In [8]:
gensim_texts = texts.apply(str.split)
dictionary = Dictionary(gensim_texts)
corpus = [dictionary.doc2bow(text) for text in gensim_texts]

In [9]:
tf_idf = TfidfModel(corpus)
corpus_tf_idf = tf_idf[corpus]

### LSI

In [10]:
from gensim.models import lsimodel

In [11]:
lsi = lsimodel.LsiModel(corpus=corpus_tf_idf, id2word=dictionary, num_topics=20)

In [12]:
lsi.show_topics(10)

[(0,
  '0.194*"карта" + 0.137*"кредит" + 0.115*"деньга" + 0.112*"вклад" + 0.108*"счёт" + 0.100*"отделение" + 0.099*"сумма" + 0.098*"заявление" + 0.098*"сотрудник" + 0.097*"мой"'),
 (1,
  '0.364*"вклад" + -0.230*"звонок" + -0.229*"номер" + 0.186*"карта" + -0.179*"ваш" + -0.161*"кредит" + -0.147*"телефон" + -0.138*"звонить" + -0.135*"данные" + -0.131*"персональный"'),
 (2,
  '-0.612*"вклад" + 0.251*"карта" + -0.213*"ставка" + -0.172*"пополнение" + 0.149*"банкомат" + -0.143*"условие" + -0.131*"договор" + 0.128*"сбербанк" + 0.095*"деньга" + -0.093*"процент"'),
 (3,
  '0.350*"страховка" + 0.340*"кредит" + 0.276*"страхование" + -0.199*"номер" + 0.163*"договор" + -0.159*"карта" + 0.138*"страховой" + -0.132*"звонок" + 0.127*"погашение" + -0.125*"вклад"'),
 (4,
  '-0.183*"средство" + -0.160*"денежный" + -0.158*"сумма" + 0.151*"заявка" + -0.150*"рубль" + -0.141*"договор" + -0.138*"платёж" + -0.133*"задолженность" + -0.133*"счёт" + -0.129*"вклад"'),
 (5,
  '0.239*"сбербанк" + 0.205*"страхование" 

Интерпретируемые темы:

1) 2 тема - комментарий, связанный с услугой вклада
2) 3 тема - комментарии по услугам страхования
3) 5 тема - комментарий по работе сбербанка

## 2. Найдите темы, в которых упомянуты конкретные банки (Сбербанк, ВТБ, другой банк). 

- Можете ли вы их прокомментировать / объяснить?

In [13]:
lsi.show_topic(5)

[('сбербанк', 0.2389898024409501),
 ('страхование', 0.20498889034919876),
 ('карта', -0.19925319204615058),
 ('заявление', 0.18178591557952412),
 ('кредитный', -0.1761846423116907),
 ('задолженность', -0.16586934366114064),
 ('страховка', 0.16056825865603586),
 ('лимит', -0.15277699102892736),
 ('платёж', -0.15068949385595676),
 ('денежный', 0.13924801603591444)]

In [14]:
lsi.show_topic(19)

[('ставка', -0.23845891129546282),
 ('р', -0.23585432199166306),
 ('услуга', 0.1805170957227242),
 ('номер', -0.1749960284552324),
 ('руб', 0.16201025418031673),
 ('персональный', 0.1608282637455236),
 ('втб', 0.14852286124140593),
 ('вклад', 0.13764264972941007),
 ('ваш', 0.13044197989045422),
 ('задолженность', -0.12320174856096977)]

In [16]:
lsi.show_topic(17)

[('руб', 0.561014590742397),
 ('комиссия', 0.2914384818968257),
 ('р', -0.29050687893578225),
 ('тыс', 0.16869480904652026),
 ('счёт', -0.13489891935558648),
 ('ваш', 0.11538762110534864),
 ('тариф', 0.10718987286195412),
 ('сбербанк', 0.09614681262062189),
 ('рубль', -0.09368855088864551),
 ('г', 0.08301405879979155)]

- Сбербанк часто попадает в набор наиболее влиятельных тематических маркеров. Может быть, "мусорное слово", т.к. упоминается очень часто
- У других банков трудно выделить какую-то закономерность